https://huggingface.co/docs/transformers/v4.42.0/ko/tasks/token_classification

In [ ]:
!pip install  evaluate
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 43.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which i

In [ ]:
!pip install -U transformers datasets==2.21.0 scipy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 93.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

# 문장 분류 모델 학습 수행

In [ ]:
import random
import logging
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric, ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

모델 지정 및 task 지정
- task 지정은 huggingface에서 제공중인 klue 데이터 세트를 다운로드 받기 위함

In [ ]:
model_checkpoint = "klue/roberta-base"
batch_size = 64
task = "ner"

In [ ]:
#['ynat', 'sts', 'nli', 'ner', 're', 'dp', 'mrc', 'wos']
datasets = load_dataset("klue", task)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/21008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

# 데이터 세트 확인

In [ ]:
# 훈련 세트와 검증 세트가 부여된 것을 확인
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 21008
    })
    validation: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 5000
    })
})

In [ ]:
# Named Entity Recongnition을 위한 태그 확인

"""
인물(PS)
장소(LC)
기관(OG)
날짜표현(DT)
시간표현(TI)
수치표현(QT)

B : 태그의 시작을 의미
I : 태그의 중간을 의미
O : 태그의 끝을 의미 ( 또는 Tag가 아닌 것. 띄어 쓰기도 여기에 포함 )

ex) 특히 영동고속도로 강릉 방향
    특히_영동고속도로_강릉_방향
    O,O,O,B-LC,I-LC,I-LC,I-LC,I-LC,I-LC,O,B-LC,I-LC,O,O,O
"""

ner_feature = datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O'], id=None), length=-1, id=None)

In [ ]:
# tag만 뽑기
label_names = ner_feature.feature.names
label_names

['B-DT',
 'I-DT',
 'B-LC',
 'I-LC',
 'B-OG',
 'I-OG',
 'B-PS',
 'I-PS',
 'B-QT',
 'I-QT',
 'B-TI',
 'I-TI',
 'O']

In [ ]:
words = datasets["train"][0]["tokens"]
labels = datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + "\t"
    line2 += full_label + "\t"

print(line1)
print(line2)

특	히	 	영	동	고	속	도	로	 	강	릉	 	방	향	 	문	막	휴	게	소	에	서	 	만	종	분	기	점	까	지	 	5	㎞	 	구	간	에	는	 	승	용	차	 	전	용	 	임	시	 	갓	길	차	로	제	를	 	운	영	하	기	로	 	했	다	.	
O	O	O	B-LC	I-LC	I-LC	I-LC	I-LC	I-LC	O	B-LC	I-LC	O	O	O	O	B-LC	I-LC	I-LC	I-LC	I-LC	O	O	O	B-LC	I-LC	I-LC	I-LC	I-LC	O	O	O	B-QT	I-QT	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	O	


In [ ]:
# 전체적인 데이터 세트 구조 확인하기. 랜덤하게 문장을 보여줌
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []

    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,sentence,tokens,ner_tags
0,앞서 지난 <30일:DT>에는 무소속 <유승민:PS> 후보의 친딸 <유담:PS> 씨가 화제가 되기도 했다.,"[앞, 서, , 지, 난, , 3, 0, 일, 에, 는, , 무, 소, 속, , 유, 승, 민, , 후, 보, 의, , 친, 딸, , 유, 담, , 씨, 가, , 화, 제, 가, , 되, 기, 도, , 했, 다, .]","[12, 12, 12, 12, 12, 12, 0, 1, 1, 12, 12, 12, 12, 12, 12, 12, 6, 7, 7, 12, 12, 12, 12, 12, 12, 12, 12, 6, 7, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]"
1,"영상 촬영자 <A:PS>씨는 이 과정에서 <B:PS>씨가 자신에게 보였던 고압적인 태도를 참을 수 없었고, 해당 영상을 백화점을 나선 지 <두 시간:TI> 만에 페이스북에 게재했다고 밝혔다.","[영, 상, , 촬, 영, 자, , A, 씨, 는, , 이, , 과, 정, 에, 서, , B, 씨, 가, , 자, 신, 에, 게, , 보, 였, 던, , 고, 압, 적, 인, , 태, 도, 를, , 참, 을, , 수, , 없, 었, 고, ,, , 해, 당, , 영, 상, 을, , 백, 화, 점, 을, , 나, 선, , 지, , 두, , 시, 간, , 만, 에, , 페, 이, 스, 북, 에, , 게, 재, 했, 다, 고, , 밝, 혔, 다, .]","[12, 12, 12, 12, 12, 12, 12, 6, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 6, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 10, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]"
2,"<광주:LC>민주화운동 충분히 안타깝고 존경스러운 분들인건 절때 변하지 않는 사실이지만, 단지 이 영화가 <광주:LC>민주화운동을 소재로 했다고 해서 무조건 평점 <10점:QT>을 받아야하는건 아니에요.","[광, 주, 민, 주, 화, 운, 동, , 충, 분, 히, , 안, 타, 깝, 고, , 존, 경, 스, 러, 운, , 분, 들, 인, 건, , 절, 때, , 변, 하, 지, , 않, 는, , 사, 실, 이, 지, 만, ,, , 단, 지, , 이, , 영, 화, 가, , 광, 주, 민, 주, 화, 운, 동, 을, , 소, 재, 로, , 했, 다, 고, , 해, 서, , 무, 조, 건, , 평, 점, , 1, 0, 점, 을, , 받, 아, 야, 하, 는, 건, , 아, 니, 에, 요, .]","[2, 3, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 2, 3, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 9, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]"
3,<마르크 빌모츠:PS> <벨기에:OG> 감독은 <후반 15분:TI> <두 장:QT>의 교체카드로 승부수를 던졌다.,"[마, 르, 크, , 빌, 모, 츠, , 벨, 기, 에, , 감, 독, 은, , 후, 반, , 1, 5, 분, , 두, , 장, 의, , 교, 체, 카, 드, 로, , 승, 부, 수, 를, , 던, 졌, 다, .]","[6, 7, 7, 7, 7, 7, 7, 12, 4, 5, 5, 12, 12, 12, 12, 12, 10, 11, 11, 11, 11, 11, 12, 8, 9, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]"
4,<정부:OG>의 무책임한 대응에 한 사람의 국민으로서 정말 많은 분노를 느꼈습니다.,"[정, 부, 의, , 무, 책, 임, 한, , 대, 응, 에, , 한, , 사, 람, 의, , 국, 민, 으, 로, 서, , 정, 말, , 많, 은, , 분, 노, 를, , 느, 꼈, 습, 니, 다, .]","[4, 5, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]"
5,"<23일:DT> 정치권에 따르면 <원:PS> 전 원장에 대한 복수의 고소·고발 사건을 수사 중인 <서울중앙지검:OG>은 최근 <원:PS> 전 원장에 대해 <법무부:OG>에 요청, 출국금지 조치한 것으로 전해졌다.","[2, 3, 일, , 정, 치, 권, 에, , 따, 르, 면, , 원, , 전, , 원, 장, 에, , 대, 한, , 복, 수, 의, , 고, 소, ·, 고, 발, , 사, 건, 을, , 수, 사, , 중, 인, , 서, 울, 중, 앙, 지, 검, 은, , 최, 근, , 원, , 전, , 원, 장, 에, , 대, 해, , 법, 무, 부, 에, , 요, 청, ,, , 출, 국, 금, 지, , 조, 치, 한, , 것, 으, 로, , 전, 해, 졌, 다, .]","[0, 1, 1, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 6, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 4, 5, 5, 5, 5, 5, 12, 12, 12, 12, 12, 6, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 4, 5, 5, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]"
6,가해자인 <B:PS>(<22:QT>) 하사 등 간부 <2명:QT>은 <지난해 11월 26일:DT> <오후 10시:TI> <인천 옹진군 백령면:LC> 한 주유소 인근 길가에서 후임인 <A:PS> 하사의 뺨과 가슴을 폭행한 혐의로 <군 검찰:OG>에 넘겨졌다.,"[가, 해, 자, 인, , B, (, 2, 2, ), , 하, 사, , 등, , 간, 부, , 2, 명, 은, , 지, 난, 해, , 1, 1, 월, , 2, 6, 일, , 오, 후, , 1, 0, 시, , 인, 천, , 옹, 진, 군, , 백, 령, 면, , 한, , 주, 유, 소, , 인, 근, , 길, 가, 에, 서, , 후, 임, 인, , A, , 하, 사, 의, , 뺨, 과, , 가, 슴, 을, , 폭, 행, 한, , 혐, 의, 로, , 군, , 검, 찰, 에, , 넘, 겨, ...]","[12, 12, 12, 12, 12, 6, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 9, 12, 12, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 10, 11, 11, 11, 11, 11, 12, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12

# Tokenization

In [ ]:
# klue/roberta-base 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# 입력 데이터 생성

# is_split_into_word=True 옵션을 이용해 단어 내의 하위 단어로 토큰 생성.
inputs = tokenizer(datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 '특',
 '히',
 '영',
 '동',
 '고',
 '속',
 '도',
 '로',
 '강',
 '릉',
 '방',
 '향',
 '문',
 '막',
 '휴',
 '게',
 '소',
 '에',
 '서',
 '만',
 '종',
 '분',
 '기',
 '점',
 '까',
 '지',
 '5',
 '㎞',
 '구',
 '간',
 '에',
 '는',
 '승',
 '용',
 '차',
 '전',
 '용',
 '임',
 '시',
 '갓',
 '길',
 '차',
 '로',
 '제',
 '를',
 '운',
 '영',
 '하',
 '기',
 '로',
 '했',
 '다',
 '.',
 '[SEP]']

In [ ]:
# 각 word별(한글은 한 글자) 아이디가 새롭게 부여된 것을 확인. is_split_into_words=True 때문에... ^^;;
inputs.word_ids()

[None,
 0,
 1,
 3,
 4,
 5,
 6,
 7,
 8,
 10,
 11,
 13,
 14,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 32,
 33,
 35,
 36,
 37,
 38,
 40,
 41,
 42,
 44,
 45,
 47,
 48,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 63,
 64,
 65,
 None]

In [ ]:
# 분해된 단어( 여기서는 음절 )에 다시 ner_tags에 있는 id를 부여함. 토큰을 재정렬
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id != current_word:
      # Start of a new word!
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)
    elif word_id is None:
        # Special token
      new_labels.append(-100)
    else:
      # Same word as previous token
      label = labels[word_id]
      # If the label is B-XXX we change it to I-XXX
      if label % 2 == 1:
          label += 1
      new_labels.append(label)

  return new_labels

In [ ]:
labels = datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()

print(labels) # 원래 문장의 레이블 확인
print(align_labels_with_tokens(labels, word_ids)) # [CLS] 토큰과 [SEP] 토큰에 -100 레이블이 붙어 있는 것을 확인

[12, 12, 12, 2, 3, 3, 3, 3, 3, 12, 2, 3, 12, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
[-100, 12, 12, 2, 3, 3, 3, 3, 3, 2, 3, 12, 12, 2, 3, 3, 3, 3, 12, 12, 2, 3, 3, 3, 3, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, -100]


In [ ]:
# 토큰 재정렬 및 klue/roberta-base 모델의 최대 길이에 맞게 문장 시퀀스 자르기
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True # truncation=True 옵션을 이용해 모델이 지원하는 최대 길이보다 커지지 않도록 설정
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
# 모든 데이터에 대해 tag 정렬 및 truncate 적용
tokenized_datasets = datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=datasets["train"].column_names,
)

Map:   0%|          | 0/21008 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

# DataLoader 구현
여기서는 입력 데이터의 labels와 훈련 데이터 토큰의 길이가 반드시 일치해야 합니다. 따라서 동적으로 패딩을 부여할 수 있는 방법을 상요합니다.

In [ ]:
# Label 확인
before_data_collator = [tokenized_datasets["train"][i]['labels'] for i in range(2)]

print(before_data_collator[0])
print(before_data_collator[1])

[-100, 12, 12, 2, 3, 3, 3, 3, 3, 2, 3, 12, 12, 2, 3, 3, 3, 3, 12, 12, 2, 3, 3, 3, 3, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, -100]
[-100, 8, 9, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, -100]


In [ ]:
from transformers import DataCollatorForTokenClassification

# 데이터의 길이게 맞게 동적으로 패딩을 주기 위함.
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,   12,   12,    2,    3,    3,    3,    3,    3,    2,    3,   12,
           12,    2,    3,    3,    3,    3,   12,   12,    2,    3,    3,    3,
            3,   12,   12,    8,    9,   12,   12,   12,   12,   12,   12,   12,
           12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,
           12,   12,   12,   12,   12,   12, -100],
        [-100,    8,    9,    9,   12,   12,   12,   12,   12,   12,   12,   12,
           12,   12,   12,   12,   12,   12,   12, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100]])

# 평가 기준 (Metric) 정의
NER Task에서의 평가는 각 토큰마다 부여된 태그의 id를 잘 맞췄는지를 예측해야 합니다. 이 때 토큰 분류를 위해 자주 사용되는 라이브러리는 `seqeval`입니다.

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d74eaaf6f1e45b8e1e013843dba53a31d2f5e682aebb6f39aa3ca268139bdb6c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
# Fake 데이터로 seqeval 테스트 해보기
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
fake_preds, fake_labels

(array([1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
        0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
        1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1]))

In [ ]:
# seqeval에 의한 metric의 compute가 잘 실행되는지 확인
labels = datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]

predictions = labels.copy()
predictions[3] = 'O'
metric.compute(predictions=[predictions], references=[labels])

{'LC': {'precision': 0.75, 'recall': 0.75, 'f1': 0.75, 'number': 4},
 'QT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 0.8,
 'overall_recall': 0.8,
 'overall_f1': 0.8000000000000002,
 'overall_accuracy': 0.9848484848484849}

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

print(id2label)
print(label2id)

{0: 'B-DT', 1: 'I-DT', 2: 'B-LC', 3: 'I-LC', 4: 'B-OG', 5: 'I-OG', 6: 'B-PS', 7: 'I-PS', 8: 'B-QT', 9: 'I-QT', 10: 'B-TI', 11: 'I-TI', 12: 'O'}
{'B-DT': 0, 'I-DT': 1, 'B-LC': 2, 'I-LC': 3, 'B-OG': 4, 'I-OG': 5, 'B-PS': 6, 'I-PS': 7, 'B-QT': 8, 'I-QT': 9, 'B-TI': 10, 'I-TI': 11, 'O': 12}


# 모델 생성 및 훈련

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.config.num_labels

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


13

In [ ]:
args = TrainingArguments(
    "test-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.162000,0.170282,0.737939,0.759627,0.748626,0.947129
2,0.103600,0.143539,0.783044,0.794908,0.788931,0.957166
3,0.073100,0.144249,0.784104,0.807533,0.795646,0.958448


TrainOutput(global_step=7878, training_loss=0.13871999596786788, metrics={'train_runtime': 421.6165, 'train_samples_per_second': 149.482, 'train_steps_per_second': 18.685, 'total_flos': 2534389622876352.0, 'train_loss': 0.13871999596786788, 'epoch': 3.0})

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/test-ner/checkpoint-7878"
token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="simple", device='cuda')
token_classifier("2013년도 강릉에서 열리는 모터쇼에 참가한 레이싱 모델 허윤미 씨의 귀요미 셀카입니다.")

[{'entity_group': 'DT',
  'score': 0.9803092,
  'word': '2013년도',
  'start': 0,
  'end': 6},
 {'entity_group': 'LC',
  'score': 0.8509516,
  'word': '강릉',
  'start': 7,
  'end': 9},
 {'entity_group': 'PS',
  'score': 0.995883,
  'word': '허윤미',
  'start': 32,
  'end': 35}]

In [ ]:
token_classifier("마! 느그 서장 남청동 살제")

[{'entity_group': 'LC',
  'score': 0.98291326,
  'word': '남청동',
  'start': 9,
  'end': 12}]